In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cryptocurrencies: a brief analysis

Cryptocurrencies have become very popular in the last years. Bitcoin is still the most popular cryptocurrency even if in the last years others crypto have emerged, for example Ethereum.
In this notebook I will compare different cryptocurrencies (Bitcoin, Dogecoin, Ethereum, Litecoin, Ripple and Monero) between themselves and with Euro/Dollar exchange rate which is the most important exchange rate in the world and gold which is traditionally considered as an asset to invest when there is uncertainty in the markets. All data have been downloaded from Yahoo finance. Data about cryptocurrencies and gold are in dollar.    

In [ ]:
Bitcoin = pd.read_csv('../input/currencies/BTC-USD.csv')
Dogecoin = pd.read_csv('../input/currencies/DOGE-USD.csv')
Ethereum = pd.read_csv('../input/currencies/ETH-USD.csv')
Euro_dollar = pd.read_csv('../input/currencies/EURUSDX.csv')
Gold = pd.read_csv('../input/currencies/GCF.csv')
Litecoin = pd.read_csv('../input/currencies/LTC-USD.csv')
Ripple = pd.read_csv('../input/currencies/XMR-USD.csv')
Monero = pd.read_csv('../input/currencies/XRP-USD.csv')

First, let's create a list with all of the currecies we want to confront.

In [ ]:
data = [Bitcoin, Dogecoin, Ethereum, Euro_dollar,Gold, Litecoin, Ripple, Monero]

Let's convert 'Date' to pandas datetime in order to plot currencies.

In [ ]:
for d in data:
    d['Date'] = pd.to_datetime(d['Date'],format = '%Y-%m-%d')
    d.set_index(d['Date'],  inplace = True)

Now we can store currencies data in a dictionary where keys are the names of the currencies and data are their respective values.

In [ ]:
currency_dict = {'Bitcoin':Bitcoin, 'Dogecoin':Dogecoin,'Ethereum':Ethereum,'Euro_dollar':Euro_dollar,'Gold':Gold,'Litecoin':Litecoin,'Ripple':Ripple,'Monero':Monero}

Available data from cryptocurrecies are different so in order to compare them in the same time horizon let's find out what is the min date for each crypto from yahoo.

In [ ]:
for d in currency_dict.keys():
    print(f"Min Date for {d} : {min(currency_dict[d].index)}")

Minimun data for each currency is 2015-08-07 so we can filter data in the dictionary starting from this date.

In [ ]:
for d in currency_dict.keys():
    currency_dict[d] = currency_dict[d][currency_dict[d].index>= '2015-08-07']

Let's plot now the graphs:

In [ ]:
fig, ax = plt.subplots(nrows = 4 , ncols = 2 , figsize =(14,14))
sns.lineplot(data = currency_dict['Bitcoin'], x = currency_dict['Bitcoin'].index, y = currency_dict['Bitcoin']['Adj Close'],ax = ax[0,0], color = 'red')
ax[0,0].set_title('Bitcoin', fontsize = 15)
sns.lineplot(data = currency_dict['Dogecoin'], x = currency_dict['Dogecoin'].index, y = currency_dict['Dogecoin']['Adj Close'],ax = ax[0,1], color = 'brown')
ax[0,1].set_title('Dogecoin', fontsize = 15)
sns.lineplot(data = currency_dict['Ethereum'], x = currency_dict['Ethereum'].index, y = currency_dict['Ethereum']['Adj Close'],ax = ax[1,0], color = 'green')
ax[1,0].set_title('Ethereum', fontsize = 15)
sns.lineplot(data = currency_dict['Euro_dollar'], x = currency_dict['Euro_dollar'].index, y = currency_dict['Euro_dollar']['Adj Close'],ax = ax[1,1], color = 'orange')
ax[1,1].set_title('Euro_dollar', fontsize = 15)
sns.lineplot(data = currency_dict['Gold'], x = currency_dict['Gold'].index, y = currency_dict['Gold']['Adj Close'],ax = ax[2,0], color = 'cyan')
ax[2,0].set_title('Gold', fontsize = 15)
sns.lineplot(data = currency_dict['Litecoin'], x = currency_dict['Litecoin'].index, y = currency_dict['Litecoin']['Adj Close'],ax = ax[2,1], color = 'purple')
ax[2,1].set_title('Litecoin', fontsize = 15)
sns.lineplot(data = currency_dict['Ripple'], x = currency_dict['Ripple'].index, y = currency_dict['Ripple']['Adj Close'],ax = ax[3,0], color = 'olive')
ax[3,0].set_title('Ripple', fontsize = 15)
sns.lineplot(data = currency_dict['Monero'], x = currency_dict['Monero'].index, y = currency_dict['Monero']['Adj Close'],ax = ax[3,1], color = 'gray')
ax[3,1].set_title('Monero', fontsize = 15)
fig.tight_layout()
plt.subplots_adjust(top=0.92)
plt.suptitle('Criptocurrencies vs Gold and EUR/USD',  fontsize = 20)
sns.despine()
plt.show()

From this graphs we can see that cryptocurrencies are much more volatile than for example Euro/dollar exchange rate. We will analize this more deeper next.

To compare different criptocurrencies, gold and EUR/USD we can cumpute daily return starting with a base value of 100 at 2015-08-07. Day_cum_return takes a daily quotations of an asset and compute daily returns on adjusted close and cumulative returns.

In [ ]:
def day_cum_returns(df):
    #compute daily and cumulative returns
    df['Daily_return'] = df['Adj Close'].pct_change()
    df['Cumulative_return'] = (1+df['Daily_return']).cumprod()*100 
    df.dropna(axis = 0, inplace = True)    
    return df['Daily_return'],df['Cumulative_return'] 

We can now plot function to every asset in currency_dict.

In [ ]:
for d in currency_dict:
    currency_dict[d]['Daily_return'],currency_dict[d]['Cumulative_return'] = day_cum_returns(currency_dict[d])
    currency_dict[d].drop('Date', axis =1, inplace = True)

We can now plot cumulative returns so we have a way to confront different asset in the same time period starting from a level of 100 at '2015-08-07.

In [ ]:
fig, ax = plt.subplots(figsize = (20,15))
sns.lineplot(data = currency_dict['Bitcoin'], x =  currency_dict['Bitcoin'].index, y =  currency_dict['Bitcoin']['Cumulative_return'], color = 'red', label = 'Bitcoin',ax=ax)
sns.lineplot(data =  currency_dict['Dogecoin'], x = currency_dict['Dogecoin'].index, y = currency_dict['Dogecoin']['Cumulative_return'], color = 'brown', label = 'Dogecoin',ax=ax)
sns.lineplot(data = currency_dict['Ethereum'], x = currency_dict['Ethereum'].index, y = currency_dict['Ethereum']['Cumulative_return'],ax = ax, color = 'green', label = 'Ethereum')
sns.lineplot(data = currency_dict['Euro_dollar'], x = currency_dict['Euro_dollar'].index, y = currency_dict['Euro_dollar']['Cumulative_return'],ax = ax, color = 'orange', label ='Euro_dollar')
sns.lineplot(data = currency_dict['Gold'], x = currency_dict['Gold'].index, y = currency_dict['Gold']['Cumulative_return'],ax = ax,label ='Gold', color = 'cyan')
sns.lineplot(data = currency_dict['Litecoin'], x = currency_dict['Litecoin'].index, y = currency_dict['Litecoin']['Cumulative_return'],ax = ax,label = 'Litecoin', color = 'purple')
sns.lineplot(data = currency_dict['Ripple'], x = currency_dict['Ripple'].index, y = currency_dict['Ripple']['Cumulative_return'],ax = ax,label='Ripple', color = 'olive')
sns.lineplot(data = currency_dict['Monero'], x = currency_dict['Monero'].index, y = currency_dict['Monero']['Cumulative_return'],ax = ax,label = 'Monero', color = 'gray')

There is a peak of volatility aroun 2018 and another peak seems to be happening in 2021 especially for Ethereum and Dogecoin and Ripple.

We can calculate cumulative return at a year level by resampling data at year level.

In [ ]:
annual_return = pd.DataFrame(columns = ['Share','Return'])

In [ ]:
for d in currency_dict.keys():
    annual_return =annual_return.append(pd.DataFrame({'Share':d, 'Return':currency_dict[d]['Daily_return'].resample('A').sum().mul(100)}))

In [ ]:
annual_return.dropna(axis = 0, inplace = True)
annual_return.reset_index(inplace = True)
annual_return.rename(columns = {'index':'Date'}, inplace = True)
annual_return['Year'] = (annual_return['Date'].dt.year).astype('object') 

In [ ]:
pd.pivot_table( data = annual_return, index ='Year', columns = 'Share')

Annual return for Dogecoin, Ripple and Ethereum are really high so far.

We can also investigate correlation between daily returns of different criptocurrencies.

In [ ]:
daily_return_df = pd.DataFrame()

In [ ]:
for d in currency_dict.keys():
    daily_return_df[d] = currency_dict[d]['Daily_return']

In [ ]:
daily_return_df.dropna(axis = 0, inplace = True)

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))
plt.title('Correlations')
sns.heatmap(daily_return_df.reset_index().corr(), annot= True,ax =ax,xticklabels=True, yticklabels=True,square = True)
ax.set_aspect("equal")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0) # rotate the x labels to be horizontally again
ax.set_yticklabels(ax.get_yticklabels(), rotation=90) # rotate the y labels to be horizontally
plt.tight_layout()

In [ ]:
daily_return_df.corr()

Cryptocurrencies seems to be quite correlated especially with Bitcoin. Euro/dollar rate is not related with others crypto whereas there is only a small correlation between gold and Bitcoin.

To investigate more volatility we can plot graphs of assets along with their 200 days moving averages and Bollinger bands. Bollinger band are calculated with the rolling standard deviation of x periods, where x need to be correlated with moving average time period. It's an indicator of volatility.


In [ ]:
for d in currency_dict:
    currency_dict[d]['SMA200'] = currency_dict[d]['Adj Close'].rolling(200).mean()
    currency_dict[d]['Bollinger_upper'] = currency_dict[d]['SMA200'] + 2*currency_dict[d]['Adj Close'].rolling(200).std()
    currency_dict[d]['Bollinger_down'] = currency_dict[d]['SMA200'] - 2*currency_dict[d]['Adj Close'].rolling(200).std()
    currency_dict[d].dropna(axis = 0, inplace = True)

In [ ]:
fig, ax = plt.subplots(nrows = 4 , ncols = 2 , figsize =(14,14))
#Bitcoin
sns.lineplot(x = currency_dict['Bitcoin'].index, y = currency_dict['Bitcoin']['Adj Close'], ax = ax[0,0] , color = 'blue', label = 'Prices')
sns.lineplot(x = currency_dict['Bitcoin'].index, y = currency_dict['Bitcoin']['SMA200'], ax = ax[0,0] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Bitcoin'].index, y = currency_dict['Bitcoin']['Bollinger_upper'], ax = ax[0,0] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Bitcoin'].index, y = currency_dict['Bitcoin']['Bollinger_down'], ax = ax[0,0] , color = 'indigo', label = 'Bollinger lower band')
ax[0,0].set_title('Botcoin volatility')
#Dogecoin
sns.lineplot(x = currency_dict['Dogecoin'].index, y = currency_dict['Dogecoin']['Adj Close'], ax = ax[0,1] , color = 'brown', label = 'Prices')
sns.lineplot(x = currency_dict['Dogecoin'].index, y = currency_dict['Dogecoin']['SMA200'], ax = ax[0,1] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Dogecoin'].index, y = currency_dict['Dogecoin']['Bollinger_upper'], ax = ax[0,1] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Dogecoin'].index, y = currency_dict['Dogecoin']['Bollinger_down'], ax = ax[0,1] , color = 'indigo', label = 'Bollinger lower band')
ax[0,1].set_title('Dogecoin volatility')
#Ethereum
sns.lineplot(x = currency_dict['Ethereum'].index, y = currency_dict['Ethereum']['Adj Close'], ax = ax[1,0] , color = 'green', label = 'Prices')
sns.lineplot(x = currency_dict['Ethereum'].index, y = currency_dict['Ethereum']['SMA200'], ax = ax[1,0] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Ethereum'].index, y = currency_dict['Ethereum']['Bollinger_upper'], ax = ax[1,0] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Ethereum'].index, y = currency_dict['Ethereum']['Bollinger_down'], ax = ax[1,0] , color = 'indigo', label = 'Bollinger lower band')
ax[1,0].set_title('Ethereum volatility')
#Euro-Dollar
sns.lineplot(x = currency_dict['Euro_dollar'].index, y = currency_dict['Euro_dollar']['Adj Close'], ax = ax[1,1] , color = 'orange', label = 'Prices')
sns.lineplot(x = currency_dict['Euro_dollar'].index, y = currency_dict['Euro_dollar']['SMA200'], ax = ax[1,1] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Euro_dollar'].index, y = currency_dict['Euro_dollar']['Bollinger_upper'], ax = ax[1,1] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Euro_dollar'].index, y = currency_dict['Euro_dollar']['Bollinger_down'], ax = ax[1,1] , color = 'indigo', label = 'Bollinger lower band')
ax[1,1].set_title('Euro_dollar volatility')
#Gold
sns.lineplot(x = currency_dict['Gold'].index, y = currency_dict['Gold']['Adj Close'], ax = ax[2,0] , color = 'cyan', label = 'Prices')
sns.lineplot(x = currency_dict['Gold'].index, y = currency_dict['Gold']['SMA200'], ax = ax[2,0] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Gold'].index, y = currency_dict['Gold']['Bollinger_upper'], ax = ax[2,0] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Gold'].index, y = currency_dict['Gold']['Bollinger_down'], ax = ax[2,0] , color = 'indigo', label = 'Bollinger lower band')
ax[2,0].set_title('Gold volatility')
#Litecoin
sns.lineplot(x = currency_dict['Litecoin'].index, y = currency_dict['Litecoin']['Adj Close'], ax = ax[2,1] , color = 'purple', label = 'Prices')
sns.lineplot(x = currency_dict['Litecoin'].index, y = currency_dict['Litecoin']['SMA200'], ax = ax[2,1] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Litecoin'].index, y = currency_dict['Litecoin']['Bollinger_upper'], ax = ax[2,1] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Litecoin'].index, y = currency_dict['Litecoin']['Bollinger_down'], ax = ax[2,1] , color = 'indigo', label = 'Bollinger lower band')
ax[2,1].set_title('Litecoin volatility')
#Ripple
sns.lineplot(x = currency_dict['Ripple'].index, y = currency_dict['Ripple']['Adj Close'], ax = ax[3,0] , color = 'olive', label = 'Prices')
sns.lineplot(x = currency_dict['Ripple'].index, y = currency_dict['Ripple']['SMA200'], ax = ax[3,0] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Ripple'].index, y = currency_dict['Ripple']['Bollinger_upper'], ax = ax[3,0] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Ripple'].index, y = currency_dict['Ripple']['Bollinger_down'], ax = ax[3,0] , color = 'indigo', label = 'Bollinger lower band')
ax[3,0].set_title('Ripple volatility')
#Monero
sns.lineplot(x = currency_dict['Monero'].index, y = currency_dict['Monero']['Adj Close'], ax = ax[3,1] , color = 'gray', label = 'Prices')
sns.lineplot(x = currency_dict['Monero'].index, y = currency_dict['Monero']['SMA200'], ax = ax[3,1] , color = 'slategray',label= 'Moving average 200 days')
sns.lineplot(x = currency_dict['Monero'].index, y = currency_dict['Monero']['Bollinger_upper'], ax = ax[3,1] , color = 'indigo',label = 'Bollinger upper band' )
sns.lineplot(x = currency_dict['Monero'].index, y = currency_dict['Monero']['Bollinger_down'], ax = ax[3,1] , color = 'indigo', label = 'Bollinger lower band')
ax[3,1].set_title('Monero volatility')
plt.tight_layout()
sns.despine()
